In [1]:
import pandas as pd




In [50]:
datas = pd.read_csv('DiabetesCleaned.csv', sep=None,engine='python')
datas

,age,mets_achieved,resting_systolic,resting_diastolic,percent_hr_achieved,hyperlipid,famhx,black,newdm10
0,61.051334,5.0,132,84,0.805031,No,No,0,1
1,58.888432,5.0,138,80,0.937888,No,Yes,1,1
2,44.646133,10.1,100,74,0.954286,Yes,Yes,0,1
3,67.548256,7.0,150,90,0.842105,Yes,No,0,1
4,42.967831,7.0,110,80,0.903955,No,Yes,0,0
...,...,...,...,...,...,...,...,...,...
16771,40.626968,10.1,110,70,0.921788,No,No,0,0
16772,52.273785,10.0,134,72,0.958333,Yes,No,0,1
16773,43.225189,10.1,136,84,0.988701,No,Yes,0,0
16774,37.681042,10.1,98,82,0.939560,No,No,0,0


In [7]:
def load_in_sklearn_format(filename,target_name):
    from bunch import Bunch
    import pandas as pd
    dataset = pd.read_csv(filename, sep=None,engine='python')
    dataset = pd.read_csv(filename, sep=None,engine='python')
    object_columns = dataset.select_dtypes(include='object')
    for column in object_columns :
        dataset[column] = dataset[column].astype('category')
    dataset=pd.get_dummies(dataset)
    feature_names = dataset.columns.values.tolist()
    feature_names.remove(target_name)
    data = dataset[feature_names].to_numpy()
    target = dataset[target_name].to_numpy()
    
    
    return Bunch(data=data, target=target, feature_names = feature_names, target_names = target_name)


In [8]:
dataset=load_in_sklearn_format('DiabetesCleaned.csv','newdm10')

In [11]:
data = dataset.data
feature_names = dataset.feature_names
y = dataset.target

In [12]:
from int_met import *

In [13]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
model = RandomForestClassifier()

In [14]:
train_data = data[1:300,:]
train_labels=y[1:300]
test_data = data[301:505,:]
test_labels=y[301:505]

In [15]:
model = model.fit(train_data,y=train_labels)

In [16]:
from sklearn.metrics import f1_score
y_pred = model.predict(test_data)
error = f1_score(test_labels,y_pred) 
error
y_pred

array([0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1])

In [17]:
#necessary imports
from pyBreakDown.explainer import Explainer
from pyBreakDown.explanation import Explanation

In [18]:
def run_PyBreakDown(model,data,observations,colnames):
    explanations=[]
    y=[]
    from pyBreakDown.explainer import Explainer
    from pyBreakDown.explanation import Explanation
    from math import exp as e
    exp = Explainer(clf=model, data=data, colnames=feature_names)
    if observations.ndim==1:
        explanation = exp.explain(observation=observations,direction="up")
        pred=explanation._final_prediction
        pred=round((e(pred)/(1+e(pred))))
        y.append(pred)
        explanations.append(explanation)
    else:
        for i in range(0,len(observations)):
            observation = observations[i]
            explanation = exp.explain(observation=observation,direction="up")
            pred=explanation._final_prediction
            pred=round((e(pred)/(1+e(pred))))
            y.append(pred)
            explanations.append(explanation)
    return explanations,y

In [19]:
def get_Coefs(explanations):
    coefs=[]
    for explanation in explanations:
        coef=[]
        for feature in explanation._attributes:
            coef.append(feature.contribution)
        coefs.append(coef)
    return coefs

In [20]:
explanations1,y1 = run_PyBreakDown(model=model,data=train_data,observations=data[500:569,:],colnames=feature_names)
explanations2,y2 = run_PyBreakDown(model=model,data=train_data,observations=data[500:569,:],colnames=feature_names)

In [21]:
coefs1 = get_Coefs(explanations1)
coefs2 = get_Coefs(explanations2)
y_test = y[500:569]
X_test = data[500:569,:]

In [22]:
calc_identity(coefs1,coefs2)

(0.0, 69, 69)

In [23]:
calc_separability(X_test, coefs1)

(69, 8, 88.40579710144928)

In [24]:
calc_compactness(coefs1)

0.4914361001317524

In [25]:
calc_similarity(coefs1,X_test)

0.5840031079527622

In [28]:
calc_stability(coefs1, y_test)

<ipython-input-26-57cbd9861d72>:10: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  ct.fit(exp)


(22, 69)

In [101]:
def calc_correctness(exps, labels, true_labels) :
    incorrect = 0
    import numpy as np
    exps=np.asarray(exps)
    labels=np.asarray(labels)
    total = labels.shape[0]
    label_values = np.unique(labels)
    unique_exps = np.unique(exps, axis=0)
    correct_exps = exps[(np.where(y1==y_test)[0])]
    unique_exps = np.unique(correct_exps, axis=0)
    similar_exps = [] 
    for exp in unique_exps:
        indexes_of_exp = np.where((exps == exp).all(axis=1))[0]
        for j in indexes_of_exp:
            if (labels[j] != labels[indexes_of_exp[0]]):
                incorrect += 1
    return (len(unique_exps)-incorrect)/(len(unique_exps))

In [66]:
(np.where(y1==y_test)[0])

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 16, 18, 19, 22, 23,
       24, 27, 28, 29, 31, 32, 34, 35, 36, 37, 39, 41, 42, 44, 45, 46, 47,
       48, 49, 51, 54, 56, 57, 58, 59, 60, 63, 64, 67, 68])

In [54]:
y1==y_test

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False, False,  True, False,
        True,  True, False, False,  True,  True,  True, False, False,
        True,  True,  True, False,  True,  True, False,  True,  True,
        True,  True, False,  True, False,  True,  True, False,  True,
        True,  True,  True,  True,  True, False,  True, False, False,
        True, False,  True,  True,  True,  True,  True, False, False,
        True,  True, False, False,  True,  True])